In [31]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM,Dropout,Bidirectional
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler,MinMaxScaler

In [52]:
PATH ='C:/Users/ckdrj/Google 드라이브/baseball/data/total/'
#PATH = 'C:/Users/LIM CHANG GEON/Google 드라이브/baseball/data/total/'
hiter16 = pd.read_csv(PATH+"2020빅콘테스트_스포츠투아이_제공데이터_팀타자_2016.csv")
pitcher16 = pd.read_csv(PATH+"2020빅콘테스트_스포츠투아이_제공데이터_팀투수_2016.csv")

In [53]:
hiter17 = pd.read_csv(PATH+"2020빅콘테스트_스포츠투아이_제공데이터_팀타자_2017.csv")
pitcher17 = pd.read_csv(PATH+"2020빅콘테스트_스포츠투아이_제공데이터_팀투수_2017.csv")
hiter18 = pd.read_csv(PATH+"2020빅콘테스트_스포츠투아이_제공데이터_팀타자_2018.csv")
pitcher18 = pd.read_csv(PATH+"2020빅콘테스트_스포츠투아이_제공데이터_팀투수_2018.csv")
hiter19 = pd.read_csv(PATH+"2020빅콘테스트_스포츠투아이_제공데이터_팀타자_2019.csv")
pitcher19 = pd.read_csv(PATH+"2020빅콘테스트_스포츠투아이_제공데이터_팀투수_2019.csv")
hiter20 = pd.read_csv(PATH+"2020빅콘테스트_스포츠투아이_제공데이터_팀타자_2020.csv")
pitcher20 = pd.read_csv(PATH+"2020빅콘테스트_스포츠투아이_제공데이터_팀투수_2020.csv")

In [54]:
hiter = pd.concat([hiter16,hiter17,hiter18,hiter19,hiter20])
del hiter16,hiter17,hiter18,hiter19,hiter20
pitcher = pd.concat([pitcher16,pitcher17,pitcher18,pitcher19,pitcher20])
del pitcher16,pitcher17,pitcher18,pitcher19,pitcher20

In [55]:
class dnn_score:
    def __init__(self,AVG,ERA,team,hiter,pitcher,mode):
        self.AVG =AVG
        self.ERA =ERA
        self.team = team
        self.hiter = hiter
        self.pitcher = pitcher
        self.mode = mode
        
    def prepro(self, hiter, pitcher): #전처리
        
        hiter['AVG']= hiter['HIT']/hiter['AB']
        hiter["y_m"]=hiter.GDAY_DS.astype(str).str[2:6]
        hiter=hiter[["T_ID","AVG",'y_m']]
        #del hiter['HIT'], hiter['AB'], hiter["GDAY_DS"]
    
        pitcher["ERA"] = 27*pitcher.ER/pitcher.INN2
        pitcher.WLS = pitcher.WLS.map({'W':1,'D':0.5,"L":0})
        pitcher["y_m"]=pitcher.GDAY_DS.astype(str).str[2:6]
        pitcher=pitcher[["T_ID","WLS","ERA","y_m"]]
        #del pitcher["ER"], pitcher["INN2"],pitcher["GDAY_DS"]
    
        return hiter, pitcher
    
    def make_data(self):
        hiter, pitcher = self.prepro(self.hiter, self.pitcher)
        team_list = ['WO','OB','NC','SK','LG','KT','LT','HT','SS','HH']
        df_mean=pd.DataFrame()
        code=0
        for team in team_list:
            code +=0.1
            pitcher_team= pitcher[pitcher.T_ID==team]
            hiter_team=hiter[hiter.T_ID==team]
            pitcher_mean = pitcher_team.groupby(['y_m']).mean()
            pitcher_mean["count"] =pitcher_team.groupby(['y_m']).count().T_ID
            hiter_mean = hiter_team.groupby(['y_m']).mean()
            df = pd.concat([pitcher_mean, hiter_mean], axis=1)
            df["team_code"] = code
            df_mean=pd.concat([df_mean,df])
    
        df_mean.reset_index(inplace=True, drop=True)
        df_mean = df_mean[df_mean["count"] > 9]
        df_mean.reset_index(inplace=True, drop=True)
        del df_mean["count"]
        
        sc_era = MinMaxScaler()
        sc_avg = MinMaxScaler()
        df_mean["AVG"]=sc_avg.fit_transform(df_mean[["AVG"]])
        df_mean["ERA"]=1-sc_era.fit_transform(df_mean[["ERA"]])

        df_mean=df_mean.sample(frac=1).reset_index(drop=True)

        if self.mode == 'train':
            x_train=df_mean.iloc[:240,1:]
            y_train=df_mean.iloc[:240,0]
            x_test=df_mean.iloc[240:,1:]
            y_test=df_mean.iloc[240:,0]
            return x_train, y_train,x_test,y_test, sc_era,sc_avg
        
        else:
            x_train=df_mean.iloc[:,1:]
            y_train=df_mean.iloc[:,0]
            
            return x_train, y_train, sc_era,sc_avg
    
    def run(self):
        if self.mode == "train":
            x_train, y_train,x_test,y_test, sc_era,sc_avg = self.make_data()
        else:
            x_train, y_train, sc_era,sc_avg = self.make_data()
        model=Sequential()
        model.add(Dense(20,activation='relu',input_dim=3))
        model.add(Dense(5,activation='relu'))
        model.add(Dense(1))
        model.compile(optimizer='adam', loss='mse')
        early_stop = EarlyStopping(monitor='val_loss', patience=3, verbose=1)
        #model.reset_states()
        model.fit(x_train, y_train, epochs=100, validation_split=0.15,callbacks=[early_stop])
        
        team_map = {'WO':0.1,'OB':0.2,'NC':0.3,'SK':0.4,'LG':0.5,'KT':0.6,'LT':0.7,'HT':0.8,'SS':0.9,'HH':1.0}

        if self.mode == 'train':
            evaluate = model.evaluate(x_test, y_test)
            pre=model.predict(x_test)
            ny_test=y_test.values.reshape(y_test.shape[0],1)
            print(evaluate)
            print(np.concatenate((pre,ny_test),axis=1))
        else:
            final_df=pd.DataFrame({"AVG":[self.AVG],"ERA":[self.ERA],"team_code":[team_map[self.team]]})
            final_df["AVG"] = sc_avg.transform(final_df[["AVG"]])
            final_df["ERA"] = 1-sc_era.transform(final_df[["ERA"]])
            win_rate= model.predict(final_df)
        
            return win_rate[0][0]

        
    

In [50]:
obj=dnn_score(0.286249, 3.9050, 'HH', hiter ,pitcher,mode='inference')
obj.run()

Train on 231 samples, validate on 41 samples
Epoch 1/100
231/231 [==============================] - 1s 2ms/step - loss: 0.1924 - val_loss: 0.1639
Epoch 2/100
231/231 [==============================] - 0s 39us/step - loss: 0.1476 - val_loss: 0.1192
Epoch 3/100
231/231 [==============================] - 0s 35us/step - loss: 0.1042 - val_loss: 0.0809
Epoch 4/100
231/231 [==============================] - 0s 39us/step - loss: 0.0668 - val_loss: 0.0518
Epoch 5/100
231/231 [==============================] - 0s 35us/step - loss: 0.0392 - val_loss: 0.0318
Epoch 6/100
231/231 [==============================] - 0s 35us/step - loss: 0.0235 - val_loss: 0.0211
Epoch 7/100
231/231 [==============================] - 0s 48us/step - loss: 0.0161 - val_loss: 0.0156
Epoch 8/100
231/231 [==============================] - 0s 61us/step - loss: 0.0121 - val_loss: 0.0126
Epoch 9/100
231/231 [==============================] - 0s 52us/step - loss: 0.0099 - val_loss: 0.0108
Epoch 10/100
231/231 [================

0.5558935

In [56]:
obj=dnn_score('na','na','na', hiter ,pitcher,mode='train')
obj.run()

Train on 204 samples, validate on 36 samples
Epoch 1/100
204/204 [==============================] - 1s 3ms/step - loss: 0.4781 - val_loss: 0.4317
Epoch 2/100
204/204 [==============================] - 0s 44us/step - loss: 0.3735 - val_loss: 0.3334
Epoch 3/100
204/204 [==============================] - 0s 34us/step - loss: 0.2839 - val_loss: 0.2531
Epoch 4/100
204/204 [==============================] - 0s 39us/step - loss: 0.2155 - val_loss: 0.1971
Epoch 5/100
204/204 [==============================] - 0s 39us/step - loss: 0.1739 - val_loss: 0.1683
Epoch 6/100
204/204 [==============================] - 0s 44us/step - loss: 0.1519 - val_loss: 0.1512
Epoch 7/100
204/204 [==============================] - 0s 44us/step - loss: 0.1369 - val_loss: 0.1371
Epoch 8/100
204/204 [==============================] - 0s 39us/step - loss: 0.1233 - val_loss: 0.1237
Epoch 9/100
204/204 [==============================] - 0s 39us/step - loss: 0.1104 - val_loss: 0.1106
Epoch 10/100
204/204 [================

204/204 [==============================] - 0s 34us/step - loss: 0.0069 - val_loss: 0.0069
Epoch 81/100
204/204 [==============================] - 0s 39us/step - loss: 0.0069 - val_loss: 0.0068
Epoch 82/100
204/204 [==============================] - 0s 39us/step - loss: 0.0068 - val_loss: 0.0068
Epoch 83/100
204/204 [==============================] - 0s 44us/step - loss: 0.0068 - val_loss: 0.0067
Epoch 84/100
204/204 [==============================] - 0s 39us/step - loss: 0.0068 - val_loss: 0.0069
Epoch 85/100
204/204 [==============================] - 0s 44us/step - loss: 0.0068 - val_loss: 0.0067
Epoch 86/100
204/204 [==============================] - 0s 39us/step - loss: 0.0067 - val_loss: 0.0069
Epoch 87/100
204/204 [==============================] - 0s 39us/step - loss: 0.0067 - val_loss: 0.0068
Epoch 88/100
204/204 [==============================] - 0s 39us/step - loss: 0.0067 - val_loss: 0.0066
Epoch 89/100
204/204 [==============================] - 0s 108us/step - loss: 0.0067 -